# 데이터 목록(행정동별 카운트)

- 총인구 : 파일데이터 이용
- 토지 공시지가 : 파일 데이터 이용
- 주차장 : API 작업
- 버스정류장 : 파일데이터 이용
- 어울링대여소 : 크롤링

In [ ]:
import requests
import numpy as np
import pandas as pd
import warnings
import re
warnings.filterwarnings(action='ignore')

from bs4 import BeautifulSoup

In [ ]:
# 도로명주소 → 지번주소로 변환
def api(addrs):
    locations = []
    for addr in addrs:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
        headers = {'Authorization':'KakaoAK 57a426ba3fb221a28bbb05da0282184d'} # api 가져오기
        place = requests.get(url, headers=headers).json()['documents']
        locations.append(place)

    # 도로명주소를 지번주소로 변환
    city = [] # 시,군
    town = [] # 동,읍,면
    for i in range(len(locations)):
        try:
            city.append(locations[i][0].get('address').get('region_2depth_name'))
            town.append(locations[i][0].get('address').get('region_3depth_name'))
        except IndexError:
#             print(i, '번째 주소 못가져옴', end='') # 지번주소 못 가져오는 행 날리기
#             print()
            city.append('없음')
            town.append('없음')
        except AttributeError:
            city.append(locations[i][0].get('road_address').get('region_2depth_name'))
            town.append(locations[i][0].get('road_address').get('region_3depth_name'))

    # 가져온 데이터 확인
    city_town = np.array([city, town]).T
    df_temp = pd.DataFrame(city_town, columns=['시군', '읍면동'])
    return df_temp

In [ ]:
def geo(i):
    # 법정동 -> 행정동
    i['행정동'] = i['행정동'].str.replace('산울동','도담동')
    i['행정동'] = i['행정동'].str.replace('해밀동','도담동')
    i['행정동'] = i['행정동'].str.replace('집현동','소담동')
    i['행정동'] = i['행정동'].str.replace('어진동','도담동')
    i['행정동'] = i['행정동'].str.replace('나성동','새롬동')
    i['행정동'] = i['행정동'].str.replace('가람동','한솔동')
    i['행정동'] = i['행정동'].str.replace('반곡동','소담동')
    i['행정동'] = i['행정동'].str.replace('합강동','소담동')
    # 분리 덜 된 경우
    i['행정동'] = i['행정동'].str.replace('연기면 세종리','연기면')
    i['행정동'] = i['행정동'].str.replace('조치원읍 신흥리','조치원읍')
    i['행정동'] = i['행정동'].str.replace('조치원읍 정리','조치원읍')
    i['행정동'] = i['행정동'].str.replace('조치원읍 번암리','조치원읍')
    i['행정동'] = i['행정동'].str.replace('조치원읍 신안리','조치원읍')

# 총인구

In [ ]:
# 데이터 불러오기
pd.set_option('display.max_row', 500)
pop = pd.read_csv('2021년 08월 말 기준 인구현황(내국인).csv',header=6)
pop.head()

,Unnamed: 0,Unnamed: 1,내국인 계,지역별\n내국인 비율,전월대비\n증 감,남,구성비(남),여,구성비(여),Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,전월인구수,전월세대수
0,NaN,계,"365,309",100.0%,945.0,"182,430",49.9%,"182,879",50.1%,"150,077",497.0,2.43,NaN,NaN,"364,364","149,580"
1,NaN,조치원읍,"43,348",11.9%,-7.0,"22,558",6.2%,"20,790",5.7%,"20,833",4.0,2.08,NaN,NaN,"43,355","20,829"
2,NaN,연기면,"2,768",0.8%,-5.0,"1,588",0.4%,"1,180",0.3%,"1,656",6.0,1.67,NaN,NaN,"2,773","1,650"
3,NaN,연동면,"3,153",0.9%,4.0,"1,657",0.5%,"1,496",0.4%,"1,751",6.0,1.80,NaN,NaN,"3,149","1,745"
4,NaN,부강면,"6,253",1.7%,34.0,"3,463",0.9%,"2,790",0.8%,"3,411",20.0,1.83,NaN,NaN,"6,219","3,391"


In [ ]:
cnt_pop = pop.iloc[1:21,[1,2]]
cnt_pop.columns = ['행정동','총인구']
cnt_pop['행정동'] = cnt_pop['행정동'].replace(r'[(0-9)]', '', regex=True).replace("'", '')
cnt_pop

,행정동,총인구
1,조치원읍,"43,348"
2,연기면,"2,768"
3,연동면,"3,153"
4,부강면,"6,253"
5,금남면,"8,974"
6,장군면,"6,994"
7,연서면,"7,437"
8,전의면,"5,559"
9,전동면,"3,408"
10,소정면,"2,355"


# 토지공시지가 (평균)

In [ ]:
# 데이터 불러오기
area=pd.read_csv('AL_36_D151_20210803_공시지가.csv',encoding='euc-kr')
area.head()

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자
0,3611010100100130001,3611010100,세종특별자치시 반곡동,1,일반,13-1,2002,7,25000,NaN,N,2021-08-03
1,3611010100100130001,3611010100,세종특별자치시 반곡동,1,일반,13-1,2003,1,28200,2003-06-30,N,2021-08-03
2,3611010100100130001,3611010100,세종특별자치시 반곡동,1,일반,13-1,2004,1,60000,2004-06-30,N,2021-08-03
3,3611010100100130001,3611010100,세종특별자치시 반곡동,1,일반,13-1,2008,1,20700,2008-05-31,N,2021-08-03
4,3611010100100130001,3611010100,세종특별자치시 반곡동,1,일반,13-1,2009,1,20700,2009-05-29,N,2021-08-03


In [ ]:
# 데이터 전처리
area['행정동'] = area['법정동명'].str.split(" ").str[1]
geo(area)

In [ ]:
# 카운트
cnt_area = area.pivot_table(index='행정동', values='공시지가', aggfunc='mean', margins=True)
cnt_area.reset_index(inplace=True)
cnt_area.columns = ['행정동','평균공시지가']
cnt_area

,행정동,평균공시지가
0,고운동,184624.920000
1,금남면,64802.108696
2,다정동,64773.308538
3,대평동,72259.091276
4,도담동,155268.774862
5,보람동,78801.262643
6,부강면,63728.444460
7,새롬동,104717.548151
8,소담동,47194.870803
9,소정면,29997.124687


# 주차장

In [ ]:
# 데이터 불러오기
df = pd.read_csv('parking_api.csv')

In [ ]:
#데이터 전처리
df_1 = df.copy()

df_1['시'] = df_1['address_name'].str.split(" ").str[0]
df_1['행정동'] = df_1['address_name'].str.split(" ").str[1]

sejong_parking = df_1[df_1['시'] == '세종특별자치시'].reset_index(drop=True)

# 주차장
ex_index = sejong_parking.query('stores.str.contains("전기차충전소|전용주차장")', engine='python').index
ex_sejong_parking = sejong_parking.drop(index = ex_index).reset_index(drop=True)

geo(ex_sejong_parking)

In [ ]:
# 카운트
cnt_parking = ex_sejong_parking.pivot_table(index='행정동', values='stores', aggfunc='count', margins=True)
cnt_parking.reset_index(inplace=True)
cnt_parking.columns = ['행정동','주차장']
cnt_parking

,행정동,주차장
0,고운동,2
1,금남면,8
2,다정동,6
3,도담동,48
4,보람동,10
5,부강면,5
6,새롬동,37
7,소담동,6
8,아름동,12
9,연기면,9


# 버스정류장

In [ ]:
# '세종도시교통공사_버스정류장_시설현황_20200713.csv' 활용하여 QGIS로 카운트함
cnt_bus = pd.read_csv('행정동별 버스 Count.csv')
cnt_bus.columns = ['행정동','버스정류장수']
cnt_bus

,행정동,버스정류장수
0,고운동,68
1,금남면,174
2,다정동,27
3,대평동,24
4,도담동,51
5,보람동,43
6,부강면,86
7,새롬동,49
8,소담동,102
9,소정면,41


# 어울링 대여소

In [ ]:
# 데이터 불러오기
webpage = requests.get("https://www.sejongbike.kr/userStationAction.do?process=stationTotalList&menu=21")
soup = BeautifulSoup(webpage.content, "html.parser")
# print(soup)

In [ ]:
# print(soup.find_all("tbody"))

In [ ]:
bike_list = soup.select(".cell_left")
# print(bike_list)

In [ ]:
# 데이터 전처리
bike_len = len(bike_list)
name = []
addr = []

for i in range(0,bike_len):
    if i % 2 == 0: # 짝수
        name.append(bike_list[i])
    else:
        addr.append(bike_list[i])

In [ ]:
bike = pd.DataFrame([ x for x in zip(name,addr)])
bike.columns = ['대여소명','주소']
bike.head()

,대여소명,주소
0,[1. 첫마을 1단지],[ 세종특별자치시 한솔동 168-38 ]
1,[2. 첫마을 3단지],[ 세종특별자치시 한솔동 168-66 ]
2,[3. 첫마을 4단지],[ 세종특별자치시 한솔동 산 2 ]
3,[4. LH세종특별본부],[ 세종특별자치시 어진동 178-286 ]
4,[5. 한솔동 주민센터],[ 세종특별자치시 한솔동 산 6 ]


In [ ]:
bike_len = bike.shape[0]
for i in range(0, bike_len):
    bike['대여소명'][i] = re.sub('(<([^>]+)>)', '',str(bike.iloc[i,0]))
    bike['주소'][i] = re.sub('(<([^>]+)>)', '',str(bike.iloc[i,1]))

In [ ]:
bike['대여소명'] = bike['대여소명'].replace(r'[(0-9.)]', '', regex=True).replace("'", '')

In [ ]:
# bike.iloc[6,1] 오타('I') 제거
bike61_len = len(bike.iloc[6,1]) - 2
bike.iloc[6,1] = bike.iloc[6,1][0:bike61_len]

In [ ]:
# bike.iloc[44:50,1] ''\r\n' 제거
for i in range(44,51):
    bike_len = len(bike.iloc[i,1]) - 4
    bike.iloc[i,1] = bike.iloc[i,1][0:bike_len]

In [ ]:
bike.head()

,대여소명,주소
0,첫마을 단지,세종특별자치시 한솔동 168-38
1,첫마을 단지,세종특별자치시 한솔동 168-66
2,첫마을 단지,세종특별자치시 한솔동 산 2
3,LH세종특별본부,세종특별자치시 어진동 178-286
4,한솔동 주민센터,세종특별자치시 한솔동 산 6


In [ ]:
# 지번주소로 변환 후, 행정동 추출
bike['행정동'] = api(bike['주소'])['읍면동']

In [ ]:
# bike.iloc[3,1] 공백 2개 → 1개(다른 행과 통일시키기 위함)
bike.iloc[3,1] = str(bike.iloc[3,1]).replace("  "," ")
bike.iloc[:,1].str.split(" ").str[2]

bike_len = bike.shape[0]
for i in range(0, bike_len):
    if bike.iloc[i,2] == '없음':
        bike.iloc[i,2] = bike.iloc[:,1].str.split(" ").str[2][i]

In [ ]:
geo(bike)

In [ ]:
# 카운트
cnt_bike = bike.pivot_table(index='행정동', values='대여소명', aggfunc='count', margins=True)
cnt_bike.reset_index(inplace=True)
cnt_bike.columns = ['행정동','어울링대여소']
cnt_bike

,행정동,어울링대여소
0,고운동,9
1,다정동,1
2,대평동,1
3,도담동,17
4,보람동,3
5,새롬동,2
6,소담동,6
7,아름동,6
8,연기면,3
9,조치원읍,9


# 데이터 조인

In [ ]:
join0 = pd.merge(cnt_pop, cnt_area, left_on='행정동', right_on='행정동', how='outer') # 인구, 공시지가
join1 = pd.merge(join0, cnt_bus, left_on='행정동', right_on='행정동', how='outer') # 버스정류장
join2 = pd.merge(join1, cnt_bike, left_on='행정동', right_on='행정동', how='outer') # 어울링
join3 = pd.merge(join2, cnt_parking, left_on='행정동', right_on='행정동', how='outer') # 주차장

In [ ]:
df = join3.drop(index=20, axis=0)
df = df.fillna(0)
df

,행정동,총인구,평균공시지가,버스정류장수,어울링대여소,주차장
0,조치원읍,"43,348",281108.759727,498.0,9.0,73.0
1,연기면,"2,768",62920.656890,60.0,3.0,9.0
2,연동면,"3,153",44754.688822,193.0,0.0,1.0
3,부강면,"6,253",63728.444460,86.0,0.0,5.0
4,금남면,"8,974",64802.108696,174.0,0.0,8.0
5,장군면,"6,994",56674.914946,114.0,0.0,7.0
6,연서면,"7,437",46307.883971,279.0,0.0,3.0
7,전의면,"5,559",28337.697861,169.0,0.0,8.0
8,전동면,"3,408",21112.929764,153.0,0.0,4.0
9,소정면,"2,355",29997.124687,41.0,0.0,0.0


In [ ]:
df.to_csv('cluster_data.csv', encoding='utf-8-sig', index=None)